## This Notebook can be used to send some (test) .wav files to the orchestrator, to test the coco pipeline.

Note: To run this code correctly you need to install git lfs (since the .wav files are stored there). The files you see in the /samples directory are just pointers.

**Install Git LFS if you haven't already**
If on mac, run:
```python
# Install git LFS
brew install git-lfs

# Set up Git LFS for your user account
git lfs install
```

**Pull the LFS files**
```python
# Navigate to the root of this directory
cd your-path-to/coco

# Pull LFS Files
git lfs pull
```

In [1]:
# Set directory, which will be scanned recursively (including subdirectories) to find all .wav files, which are then appended to a list with all .wav-File Paths

import os

directory = "samples/"
wav_files = []

for root, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith(".wav"):
            wav_files.append(os.path.join(root, file))


In [ ]:
!pip install requests -q

In [2]:
# Function to check the status of the orchestrator and return the status
import requests
def check_status():
    url = "http://localhost:3030/status"
    headers = {
        "X-API-KEY":
        "local"
    }   
    response = requests.get(url, headers=headers)

    return response.json()

check_status()

{'status': 'success',
 'active_tasks': 0,
 'max_tasks': 1,
 'available_capacity': 1}

In [3]:
def upload_audio_file(file_path, api_key="local", base_url="http://localhost:3030"):
    """
    Upload an audio file to the orchestrator service.
    
    Args:
        file_path (str): Path to the audio file to upload
        api_key (str): API key for authentication (default: "test")
        base_url (str): Base URL of the service (default: "http://localhost:8000")
        
    Returns:
        requests.Response: HTTP response from the server
    """
    # Extract filename from path
    filename = os.path.basename(file_path)
    
    # Set headers
    headers = {
        "X-API-Key": api_key,
        "Content-Disposition": f'attachment; filename="{filename}"'
    }
    
    # Read file content
    with open(file_path, "rb") as file:
        file_content = file.read()
    
    # Send request
    response = requests.post(
        f"{base_url}/uploadAudio",
        headers=headers,
        data=file_content
    )
    
    return response

In [4]:
# Sort Files by session id and index

def sort_files(wav_files):
    sorted_files = []
    for file in wav_files:
        try:
            file_name = file.split("/")[-1]
            session_id = file_name.split("_")[0]
            index = file_name.split("_")[1]
            sorted_files.append((session_id, index, file))
        except:
            pass

    # Sort files but account that index 2 comes after 1 and not after 19
    sorted_files.sort(key=lambda x: (x[0], int(x[1])))
    return sorted_files

In [5]:
sorted_files = sort_files(wav_files)

# Print first 5 sorted files
for i in range(5):
    print(sorted_files[i])

('1', '1', 'samples/1_1_25-04-17_18-06-57_start.wav')
('1', '2', 'samples/1_2_25-04-17_18-07-07_middle.wav')
('1', '3', 'samples/1_3_25-04-17_18-07-17_middle.wav')
('1', '4', 'samples/1_4_25-04-17_18-07-27_middle.wav')
('1', '5', 'samples/1_5_25-04-17_18-07-37_middle.wav')


## Note: The execution of the later cell might take a while, depending on your system and the whisper model you've chosen.

In [10]:
# Upload all .wav files to the orchestrator but before sending an HTTP request, check the status of the orchestrator

import time

sorted_files = sort_files(wav_files)

for i,file_path in enumerate(sorted_files):

    while(check_status()["available_capacity"] == 0):
        # Optionally, you can print a message to inform the user
        # print("Orchestrator is busy. Please try again later.")
        # Wait for half a second
        time.sleep(0.5)

    file_path = file_path[2]

    response = upload_audio_file(file_path)
    print(f"Uploaded file {i+1}/{len(wav_files)}: {file_path} - Status: {response.status_code}")

    # wait for half a second
    time.sleep(0.5) 


Uploaded file 1/81: samples/1_1_25-04-17_18-06-57_start.wav - Status: 200
Uploaded file 2/81: samples/1_2_25-04-17_18-07-07_middle.wav - Status: 200
Uploaded file 3/81: samples/1_3_25-04-17_18-07-17_middle.wav - Status: 200
Uploaded file 4/81: samples/1_4_25-04-17_18-07-27_middle.wav - Status: 200
Uploaded file 5/81: samples/1_5_25-04-17_18-07-37_middle.wav - Status: 200
Uploaded file 6/81: samples/1_6_25-04-17_18-07-47_middle.wav - Status: 200
Uploaded file 7/81: samples/1_7_25-04-17_18-07-57_middle.wav - Status: 200
Uploaded file 8/81: samples/1_8_25-04-17_18-08-07_middle.wav - Status: 200
Uploaded file 9/81: samples/1_9_25-04-17_18-08-17_middle.wav - Status: 200
Uploaded file 10/81: samples/1_10_25-04-17_18-08-27_middle.wav - Status: 200
Uploaded file 11/81: samples/1_11_25-04-17_18-08-37_middle.wav - Status: 200
Uploaded file 12/81: samples/1_12_25-04-17_18-08-47_middle.wav - Status: 200
Uploaded file 13/81: samples/1_13_25-04-17_18-08-57_middle.wav - Status: 200
Uploaded file 14/8